# Double Basis
Mainly for debuging & testing

In [1]:
# Import package
using LinearAlgebra, Plots, Test
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

## Transformation
Map the vector in `TranslationalBasis` to `TensorBasis`, and check entanglement entropy.

In [2]:
@testset "random" begin
    L = 6
    rm = randn(ComplexF64, 9, 9) |> Hermitian |> Array
    B0 = TensorBasis(L=L, base=3)
    H0 = trans_inv_operator(rm, 2, B0)
    E, V = Array(H0) |> Hermitian |> eigen
    S = [ent_S(V[:, i], 1:L÷2, B0) for i in axes(V, 2)]

    E2 = Float64[]
    V2 = Vector{ComplexF64}[]
    S2 = Float64[]
    B2 = Any[]
    for i = 0:L-1
        B = TranslationalBasis(k=i, L=L, base=3)
        H = trans_inv_operator(rm, 2, B)
        e, v = Array(H) |> Hermitian |> eigen
        append!(E2, e)
        for j in axes(v, 2)
            push!(V2, v[:, j])
            push!(S2, ent_S(v[:, j], 1:L÷2, B))
            push!(B2, B)
        end
    end
    perm = sortperm(E2)
    B2 = B2[perm]
    E2 = E2[perm]
    V2 = V2[perm]
    S2 = S2[perm]
    @test E ≈ E2
    @test S ≈ S2
    
    for i in eachindex(E)
        v0 = DoubleBasis(B0, B2[i])(V2[i])
        @test norm(v0) ≈ 1.0
        @test abs(dot(v0, V[:, i])) ≈ 1.0
    end
end

Test Summary: | Pass  Total  Time
random        | 1460   1460  5.2s


Test.DefaultTestSet("random", Any[], 1460, false, false, true, 1.686030836771116e9, 1.686030841957087e9, false)

## Projector
Consider a spin-1/2 translational invariant random Hamiltonian 
$$
H = \sum_i h_{i,i+1}
$$
The Hamiltonian is irreducible in the $k=0$ sector.

If we devided the Hilbert space further to $k=0, N=0,\dots,L$, the Hamiltonian will have off diagonal blocks $H_{mn}$. 

In [3]:
L = 10
h0 = rand(ComplexF64, 4, 4) |> Hermitian
bases = [ProjectedBasis(L=L, N=i) for i in 0:L]
blocks = Matrix{Any}(undef, L+1, L+1)
for m in 1:L+1, n in 1:L+1 
    basis = DoubleBasis(bases[m], bases[n])
    blocks[m,n] = trans_inv_operator(h0, 2, basis)
end

In above, each `block[m,n]` is the $H_{mn}$. We can glue the blocks together to check its correctness by examine the eigenvalues:

In [4]:
dims = [size(b, 1) for b in bases]
mat = zeros(ComplexF64, 2^L, 2^L)
n=0
for i = 1:L+1 
    m = 0
    for j = 1:L+1
        EDKit.addto!(view(mat, n+1:n+dims[i], m+1:m+dims[j]), blocks[i,j])
        m += dims[j]
    end
    n += dims[i]
end

# original matrix
mat0 = trans_inv_operator(h0, 2, L)

# Eigen values
vals0 = eigvals(Hermitian(mat0))
vals = eigvals(Hermitian(mat))
vals - vals0 |> norm

8.708950177984901e-13

## Transformation
Consider mapping a state in ($N,k$) sector to tensor-product basis, and then the reverse map from tensor basis back to translational basis.

In [5]:
printdgt!(B, i) = (change!(B, i); print("|"); print.(B.dgt); print("⟩; "))
function showinfo(;L, k, N=nothing, p=nothing)
    B0 = TensorBasis(L=L)
    B1 = if isnothing(N) 
        isnothing(p) ? TranslationalBasis(L=L, k=k) : TranslationParityBasis(L=L, k=k, p=p)
    else
        isnothing(p) ? TranslationalBasis(L=L, N=N, k=k) : TranslationParityBasis(L=L, N=N, k=k, p=p)
    end
    dim = size(B1, 1); println("Dimension = $dim")
    for x in 1:dim  # check all basis states in `B1`
        println("--------------------------------------------------")
        v = zeros(dim); v[x] = 1; print("Ψ$x = "); printdgt!(B1, x); print("\n")
        v0 = DoubleBasis(B0, B1)(v)  # Map to tensor basis

        print("Basis: ")  # Print the results
        angles = Int64[]
        for i in eachindex(v0) 
            iszero(v0[i]) && continue
            printdgt!(B0, i)
            push!(angles, round(Int, angle(v0[i])*L/2/π))
        end
        print("\nAngle: θ₀ * ", angles, "\n")
        println("Norm: $(norm(v0))")
    
        v1 = DoubleBasis(B1, B0)(v0)  # Map to translational basis 
        print("V1 = ")
        for i in eachindex(v1) 
            iszero(v1[i]) ? continue : print("$(v1[i]) * ")
            printdgt!(B1, i)
        end
        print("\n")
    end
end
showinfo(L=6, N=3, k=1)

Dimension = 3
--------------------------------------------------
Ψ1 = |000111⟩; 
Basis: |000111⟩; |001110⟩; |011100⟩; |100011⟩; |110001⟩; |111000⟩; 
Angle: θ₀ * 

[0, 1, 2, -1, -2, 3]
Norm: 1.0
V1 = 

1.0 - 2.4492935982947064e-16im * |000111⟩; 
--------------------------------------------------
Ψ2 = |001011⟩; 
Basis: |001011⟩; |010110⟩; |011001⟩; |100101⟩; |101100⟩; |110010⟩; 
Angle: θ₀ * [0, 1, 3, -1, 2, -2]
Norm: 1.0
V1 = -0.4999999999999991 + 0.866025403784439im * |001011⟩; 
--------------------------------------------------
Ψ3 = |001101⟩; 
Basis: |001101⟩; |010011⟩; |011010⟩; |100110⟩; |101001⟩; |110100⟩; 
Angle: θ₀ * [0, -2, 1, -1, 3, 2]
Norm: 1.0
V1 = -0.5000000000000003 - 0.8660254037844385im * |001101⟩; 


In [6]:
B1 = TranslationalBasis(L=4, N=1, k=1)

TranslationalBasis{Int64, ComplexF64}([0, 0, 0, 0], [2], [2.0], ComplexF64[1.0 - 0.0im, 6.123233995736766e-17 - 1.0im, -1.0 - 1.2246467991473532e-16im, -1.8369701987210297e-16 + 1.0im], 1, 2)

In [10]:
Threads.nthreads()

16

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------
# Test Translational PXP
#-------------------------------------------------------------------------------------------------------------------------
@testset "Multi-threads Translational PXP" begin
    L, k, p = 28, 0, 1
    mat = begin
        P = Diagonal([1, 1, 1, 0, 1, 1, 0, 0])
        X = [0 1; 1 0]
        P * kron(I(2), X, I(2)) * P
    end
    pxpf(v::Vector{<:Integer}) = all(v[i]==0 || v[mod(i, length(v))+1]==0 for i=1:length(v))
    println("--------------------------------------")
    print("Single-threads:")
    @time bs = TranslationalBasis(f=pxpf, k=k, L=L, threaded=false)
    print("Multi-threads :")
    @time bm = TranslationalBasis(f=pxpf, k=k, L=L, threaded=true)
    @test bs.I == bm.I
    @test norm(bs.R-bm.R) ≈ 0.0
end

#-------------------------------------------------------------------------------------------------------------------------
# Test Translational Parity PXP
#-------------------------------------------------------------------------------------------------------------------------
@testset "Multi-threads Translational Parity PXP" begin
    L, k, p = 28, 0, 1
    mat = begin
        P = Diagonal([1, 1, 1, 0, 1, 1, 0, 0])
        X = [0 1; 1 0]
        P * kron(I(2), X, I(2)) * P
    end
    pxpf(v::Vector{<:Integer}) = all(v[i]==0 || v[mod(i, length(v))+1]==0 for i=1:length(v))
    println("--------------------------------------")
    print("Single-threads:")
    @time bs = TranslationParityBasis(f=pxpf, k=k, p=p, L=L, threaded=false)
    print("Multi-threads :")
    @time bm = TranslationParityBasis(f=pxpf, k=k, p=p, L=L, threaded=true)
    @test bs.I == bm.I
    @test norm(bs.R-bm.R) ≈ 0.0
end


In [11]:
L, k, p = 28, 0, 1
mat = begin
    P = Diagonal([1, 1, 1, 0, 1, 1, 0, 0])
    X = [0 1; 1 0]
    P * kron(I(2), X, I(2)) * P
end
pxpf(v::Vector{<:Integer}) = all(v[i]==0 || v[mod(i, length(v))+1]==0 for i=1:length(v))

pxpf (generic function with 1 method)

In [12]:
bs = TranslationalBasis(f=pxpf, k=k, L=L, threaded=false)

TranslationalBasis{Int64, Float64}([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 6, 10, 18, 22, 34, 38, 42, 66  …  43330198, 43338326, 43338390, 43341142, 44386966, 44389718, 44651862, 44717398, 44733782, 89478486], [28.0, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805  …  5.2915026221291805, 7.483314773547883, 5.2915026221291805, 5.2915026221291805, 10.583005244258361, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 19.79898987322333], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1, 2)

In [13]:
bm = TranslationalBasis(f=pxpf, k=k, L=L, threaded=true)

TranslationalBasis{Int64, Float64}([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 6, 10, 18, 22, 34, 38, 42, 66  …  43330198, 43338326, 43338390, 43341142, 44386966, 44389718, 44651862, 44717398, 44733782, 89478486], [28.0, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805  …  5.2915026221291805, 7.483314773547883, 5.2915026221291805, 5.2915026221291805, 10.583005244258361, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 5.2915026221291805, 19.79898987322333], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1, 2)